# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [2]:
!pip install python-dotenv

In [3]:
from dotenv import load_dotenv
import os 

load_dotenv()
base_url = os.getenv('BASE_URL')
api_key = os.getenv('API_KEY')
openai_api_key = os.getenv('openai_api_key')

In [7]:
from openai import OpenAI

client = OpenAI(base_url=base_url, api_key=api_key)

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一个宁静而美丽的自然景观，画面中心是一条木制的栈道，引人向前走去探索。栈道两旁长满了郁郁葱葱的绿色草地，草地上散布着一些灌木和树木。天空是蓝色的，散布着几朵白云，整体给人一种清新宁静的感觉。这种场景通常适合散步、放松或者进行自然观察，是典型的户外休闲活动场所。整体色彩鲜明，光照充足，显示出一种平和而舒适的氛围。', role='assistant', function_call=None, tool_calls=None))


In [8]:
response.choices[0].message.content

'这幅图展示了一个宁静而美丽的自然景观，画面中心是一条木制的栈道，引人向前走去探索。栈道两旁长满了郁郁葱葱的绿色草地，草地上散布着一些灌木和树木。天空是蓝色的，散布着几朵白云，整体给人一种清新宁静的感觉。这种场景通常适合散步、放松或者进行自然观察，是典型的户外休闲活动场所。整体色彩鲜明，光照充足，显示出一种平和而舒适的氛围。'

### 封装成一个函数 query_image_description

In [9]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(base_url=base_url, api_key=api_key)  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [10]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默的表示，通过修图手段，将狗的头部与人的身体结合，创造出两种不同风格的狗狗角色来形象地讲述“16岁的我”与“工作后的我”。左边的狗狗被设计成了一个非常健壮的身体，代表16岁时的雄心和活力；而右边的狗狗则表现得较为憔悴和无力，代表进入职场后的疲惫和负担。此图旨在以幽默的形式反映出人从青春期到成年后在生活压力下的变化和对比。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [14]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(api_key=openai_api_key)  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [15]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图表显示了1980年至2020年期间美国、中国、日本和德国的国内生产总值（GDP）的比较。每条线代表一个国家的GDP趋势，单位为万亿美元。

- 蓝色线表示美国，可以看到从1980年开始，美国的GDP稳步增长。
- 红色线表示中国，自1990年代后期以来，中国的GDP快速增长，尤其是进入21世纪后增长更为显著。
- 紫色线表示日本，其GDP在1990年代初期达到高峰后，增长放缓，进入21世纪后有所回落。
- 绿色线表示德国，其GDP增长相对稳定，增幅逐渐。

这张图表反映了这四个经济体在过去四十年的经济表现和增长趋势，显示出中国的快速崛起以及其他国家的相对经济状况。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [16]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图显示的是一个笔记本上的手写笔记，内容涉及了深度学习中的几种调优（tuning）策略，尤其是围绕自然语言处理(NLP)模型的调整。以下是笔记内容的简要说明：

1. 有关Prompt Tuning（提示调整）的内容，提到这是一个用于“Few-shot”和小型模型的技术。笔记描述了输入向量X由一个嵌入向量序列组成，用于Transformer的Encoder/Decoder模型，并提出输出Y=WX的计算公式，其中W代表权重矩阵。

2. 提到了Prefix Tuning（前缀调整），这是另一种调整方式，涉及调整权重矩阵W的子集W_p来改进模型性能，并用改进的权重矩阵W'和输入X'来进行预测。

3. 接着讨论了LoRA（低秩逼近）调整技术，表达式(Y = (W + ΔW)X)用于表明模型通过调整ΔW（权重的低秩更新矩阵？）来优化输出Y。同时提到了ΔW = AB的形式，展示了其具体的矩阵维度变换。

4. 最后一部分有关Q-LoRA（可能是一种LoRA的变种）的讨论，以及用于比较内存使用的数据（例如，LAMA使用65GB，而Q-LoRA使用48GB）。

这笔记整体上反映出复杂的深度学习和NLP模型调整技术，尤其关注于减少计算复杂性和提升模型在小数据集上的性能。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [19]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示了一张纸上的英文笔记。笔记内容是：“The trees were laden with apples.” 下方有一个日期：“Liangbe 6, 2024-07-21。”内容描述了某个时间点（2024年7月21日）树木挂满了苹果，这可能是一个观察日记或者个人日记的一部分。图片背景部分显示有电路板图案的材质，这可能是笔记本的封面或背景。

![](./images/handwriting_1.jpg)

In [12]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本的两页，内容涉及深度学习、特别是关于自然语言处理（NLP）的各种技术和方法。主要讨论了Transformer模型及其改进方法和训练技术。

左侧页面的上部标注有“自然语言处理”、“基础”和“评价”，可能是对内容的分类。提到了Transformer模型，并列举了不同的测试标准和指标，如PeFT (“Prompt-based Fine-Tuning”) 和模型性能对比（“Benchmark”）。此外，还提到了不同的方法，如Prompt Tuning和Adapter。具体包括：  
- Adapter: 一个2019年Google的研究
- Prefix: 代表2021年Stanford的工作
- Prompt: 同样是2021年Google的研究
- P-Tuning V1和V2：2021年的两种方法
- Soft prompts：2021年的研究，提示模板基于模板

右侧页面讨论了多模态指令式微调（multi-modality instruction FT）、Llama (3B)、LoRA、PETC（2022年的新技术）等。还有部分文字描述了如何使用prefix-tuning和Adapter方法来细化在大型语言模型（LLMs）中的处理。

页面提到了几种语言模型，如：
- Llama 
- BLOOM
- ChatGLM 
- Alpaca

这些内容表明这本笔记本的主人正在研究或学习NLP领域的最新技术和方法，特别是如何通过各种微调技术提升已有的大型语言模型的性能。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

![image_liangbe6](./images/gpt4-v.jpg)

In [17]:
content = query_base64_image_description("./images/gpt4-v.jpg")
print(content)

这张图片显示了一张纸上的英文笔记。笔记内容是：“The trees were laden with apples.” 下方有一个日期：“Liangbe 6, 2024-07-21。”内容描述了某个时间点（2024年7月21日）树木挂满了苹果，这可能是一个观察日记或者个人日记的一部分。图片背景部分显示有电路板图案的材质，这可能是笔记本的封面或背景。


In [20]:
display(Markdown(content))

这张图片显示了一张纸上的英文笔记。笔记内容是：“The trees were laden with apples.” 下方有一个日期：“Liangbe 6, 2024-07-21。”内容描述了某个时间点（2024年7月21日）树木挂满了苹果，这可能是一个观察日记或者个人日记的一部分。图片背景部分显示有电路板图案的材质，这可能是笔记本的封面或背景。

In [21]:
def query_base64_image_description_v2(image_path, max_tokens=1000):
    content = query_base64_image_description(image_path, prompt="使用markdown 格式输出你对图片内容的分析", max_tokens=max_tokens)
    display(Markdown(content))

query_base64_image_description_v2("./images/gpt4-v.jpg")

### 图片内容分析

#### 文字描述
- **文本内容**：The trees were laden with apples.
- **署名**：Liangbe 6.
- **日期**：2024-07-21.

#### 背景和其他元素
- **背景**：黑色背景，上面有白色的电子线路图案。
- **纸张位置**：黄色的纸张部分位于图片的下方，与黑色电子线路图案背景形成对比。

#### 其他观察点
- 文字和签名使用的是黑色墨水，书写清晰。
- 文本表达的是一种自然景象，即树木挂满苹果。

### 总结
此图片主要展现的是一段手写的文本，描述了一个自然景象，并提供了作者署名和具体日期。背景的电子线路图案为图片增添了技术感，与自然景象描述形成了一种独特的对比。